In [1]:
from ultis1 import *
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo
import matplotlib.pyplot as plt
from docplex.mp.model_reader import ModelReader
%matplotlib widget

### Create functions to define quadratic model and convert to qubo

In [2]:
def define_variables(costs, model):
    var = set()
    for ijk in costs.keys():
        i, j, k = ijk[0], ijk[1], ijk[2]
        var.add((i, j))
        var.add((j, k))
    var = sorted(var, key=lambda x: (x[0], x[1]))
    x = model.binary_var_dict(var, name='x')
    return x

In [3]:
def create_constrained_quadratic_model(costs, hits, hits_by_layers, alpha):
    NL = len(hits_by_layers.keys())
    print("Number of layers: ", NL)

    model = Model(name="Track finding")
    model.float_precision = 8
    x = define_variables(costs, model)
    ob_funct = model.sum(-w * alpha * x[(ijk[0], ijk[1])] * x[(ijk[1], ijk[2])] for ijk, w in costs.items())
    model.minimize(ob_funct)
    for h in hits:
        k = h.index
        constraint_out = []
        constraint_in = []
        for k_1 in x.keys():
            if ((h.layer_id < (NL - 1)) and (k_1[0] == k)):
                constraint_out.append(x[(k_1[0], k_1[1])])
            if ((h.layer_id > 0) and (k_1[1] == k)):
                constraint_in.append(x[(k_1[0], k_1[1])])
        if (len(constraint_out) > 0):
            model.add_constraint(model.sum(constraint_out) == 1)
        if (len(constraint_in) > 0):
            model.add_constraint(model.sum(constraint_in) == 1)
    return model


In [4]:
def convert_to_qubo(model, gamma, path):
    quadratic_program = from_docplex_mp(model)
    qubo = QuadraticProgramToQubo(penalty=gamma).convert(quadratic_program)
    num_vars = qubo.get_num_binary_vars()
    print(
        f"To represent the inital problem with {model.number_of_binary_variables} variables, the QUBO representation needs {num_vars} variables")

    qubo.write_to_lp_file(path)
    # model = cplex.Cplex(path)
    model = ModelReader.read(path)
    return model


In [5]:
def create_model(costs, hits, hits_by_layers, alpha, gamma, path):
    model = create_constrained_quadratic_model(costs, hits, hits_by_layers, alpha)
    model = convert_to_qubo(model, gamma, path)
    return model

In [6]:
def solve_lp(model, solution_path_out):
    model.solve(log_output=True)
    
    if model.solution == None:
        print("No solution!")
        return None
    else:
        model.solution.export(solution_path_out)

        f = open(solution_path_out)
        result = json.load(f)
        f.close()
        solution = result['CPLEXSolution']['variables']
        ob_value = float(result['CPLEXSolution']['header']['objectiveValue'])
        return ob_value, solution

### Display result

In [7]:
def display(hits, result, out=""):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    xs = []
    ys = []
    zs = []

    for h in hits:
        xs.append(h.x)
        ys.append(h.y)
        zs.append(h.z)
    ax.scatter(xs, ys, zs, marker='o', color='red')

    count_segments = 0
    for var in result:
        # print(var)
        x_i_j = var['name'].split('_')
        if 'x' in x_i_j[0] and round(float(var['value'])) == 1.0:
            count_segments += 1
            i = int(x_i_j[1])
            j = int(x_i_j[2])
            h1 = hits[i]
            h2 = hits[j]
            ax.plot(xs=[h1.x, h2.x], ys=[h1.y, h2.y], zs=[h1.z, h2.z], color='blue')
    print("No. segments by model :", count_segments)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    plt.savefig(out)
    plt.show()


In [8]:
def run_script(no_tracks, beta_max, m, alpha, gamma):
    src_path = '../../src/data_selected'
    folder = '/'+ str(no_tracks)+'hits/known_track/'
    out_path = 'results'
    data_path = src_path + folder + 'hits.csv'
    figure_path_out = out_path + folder + "result_C_QUBM.PNG"
    check_path(out_path + folder)
    
    # read data
    print("Loading data...")
    start = time.time()
    hits_by_layers, hits = read_hits(data_path)
    end = time.time()
    print("Loaded data! Execution time: ", end - start)

    re_calculate = False
    costs_path_out = out_path + folder + "pi_" + str(A) + "costs.json"
    if os.path.exists(costs_path_out) == False:
        re_calculate = True
    if re_calculate:
        # calculate costs
        print("\n----Compute cost----")
        start = time.time()
        costs = get_costs_tensorflow(hits_by_layers, beta_max)
        end = time.time()
        print('Complete!. Execution time: ', end - start, 's')
    
        print("\n---Write cost out---")
        print("Path: ", costs_path_out)
        write_costs(costs, costs_path_out, m)
    
    # load data
    print("---Load cost---")
    costs = load_costs(costs_path_out)
    print("---Loaded cost---")

    print("\n ----Create LP----")
    model_path_out = out_path + folder + "model_C_QUBM.lp"
    model = create_model(costs, hits, hits_by_layers, alpha, gamma, model_path_out)
    print("----Created LP----")

    
    print("\n----Solving----")
    solution_path_out = out_path + folder + "solution_C_QUBM.json"
    ob_value, solution = solve_lp(model, solution_path_out)
    print("Objective value:", ob_value / alpha)
    cal_expected_value(hits, m)
    display(hits, solution, out=figure_path_out)

In [9]:
if __name__ == '__main__': # Keep this line if we want to run multiprocessing
    no_tracks = 100
    A = 250
    beta_max = math.pi / A
    m = 1
    alpha = 200
    gamma = 1
    run_script(no_tracks, beta_max, m, alpha, gamma)

Loading data...
Loaded data! Execution time:  0.011720895767211914

----Compute cost----


Computing…:   0%|                                    | 0/5 [00:01<?, ?it/s]


TypeError: in user code:

    File "/Users/doduydao/daodd/PycharmProjects/track/src/Bogdan_idea/ultis1.py", line 51, in compute_cost_tensorflow  *
        costs = tf.TensorArray(((tf.int8, tf.int8, tf.int8), tf.float32), size=0, dynamic_size=True)

    TypeError: Cannot convert the argument `type_value`: ((tf.int8, tf.int8, tf.int8), tf.float32) to a TensorFlow DType.


In [ ]:
plt.close()